# Анализ прибыльности региона для добычи нефти

**Заказчик** - добывающая компания «ГлавРосГосНефть».

**Цель** -   Необходимо определить в каком из трех регионов бурить новые скважины.

**Входные данные от Заказчика**: 

Предоставлены пробы нефти в трёх регионах: в каждом 100 000 месторождений, где измерили качество нефти и объём её запасов. Необходимо построить модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль, и наименьшую вероятность убытка. Проанализируем возможную прибыль и риски техникой Bootstrap.

- Данные геологоразведки трёх регионов находятся в файлах:

  - `/datasets/geo_data_0.csv`
  
  - `/datasets/geo_data_1.csv`
  
  - `/datasets/geo_data_2.csv`


#### Описание данных

 Известно:
 
`id` — уникальный идентификатор скважины;

`f0, f1, f2` — три признака точек (неважно, что они означают, но сами признаки значимы);

`product` — объём запасов в скважине (тыс. баррелей).

#### Условия:

Для обучения модели подходит только линейная регрессия (остальные — недостаточно предсказуемые).

При разведке региона исследуют 500 точек, из которых с помощью машинного обучения выбирают 200 лучших для разработки.
Бюджет на разработку скважин в регионе — 10 млрд рублей.

При нынешних ценах один баррель сырья приносит 450 рублей дохода. Доход с каждой единицы продукта составляет 450 тыс. рублей, поскольку объём указан в тысячах баррелей.

После оценки рисков нужно оставить лишь те регионы, в которых вероятность убытков меньше 2.5%. Среди них выбирают регион с наибольшей средней прибылью.

#### Для достижения поставленной цели необходимо выполнить следующие шаги:

 - В каждом регионе собрать характеристики для скважин: качество нефти и объём её запасов;

 - Построить модель для предсказания объёма запасов в новых скважинах для каждого региона;
 
 - Выполнить эксперимент с помощью технологии Bootstrap: выбрать топ-200 из 500 выбранных наугад скважин с самыми высокими оценками значений. Эксперимент проводится 1000 раз;
 
 - Посчитать распределение прибыли от разработки. Найти среднее значение прибыли от разработки и 95%-й доверительный интервал для каждого региона;
 
 - Расчитать вероятность убыточности разработки для каждого региона;
 
 - Определить регион с минимальной вероятностью (меньше 2.5%) убыточности и максимальной суммарной прибылью отобранных скважин.

### План исследования:

- Изучение файлов с данными и Предобработка данных
- Обучение и проверка модели для каждого региона:
     - Разбить данные на обучающую и валидационную выборки в соотношении 75:25.
     - Обучение модели и формулирование предсказания на валидационной выборке.
     - Сохранение предсказания и правильные ответы на валидационной выборке.
     - Напечатать на экране средний запас предсказанного сырья и RMSE модели.
     - Анализ результатов.
- Подготовка к расчёту прибыли:
     - Все ключевые значения для расчётов сохранить в отдельных переменных.
     - Рассчитать достаточный объём сырья для безубыточной разработки новой скважины. Сравнить полученный объём сырья со средним запасом в каждом регионе.
     - Описать выводы по этапу подготовки расчёта прибыли.
- Создать функцию для расчета прибыли и рисков  по выбранным скважинам и предсказаниям модели:
     - Выбрать скважины с максимальными значениями предсказаний.
     - Просуммировать целевое значение объёма сырья, соответствующее этим предсказаниям.
     - Рассчитать прибыль для полученного объёма сырья.
- Расчет рисков и прибыли для каждого региона:
     - Применить технику Bootstrap с 1000 выборок, чтобы найти распределение прибыли.
     - Найти среднюю прибыль, 95%-й доверительный интервал и риск убытков. Убыток — это отрицательная прибыль.
     - Написать выводы: предложите регион для разработки скважин и обосновать выбор.

### Изучение файлов с данными и предобработка

Начнем изучение наших данных с импортирования необходимых для выполнения исследования библиотек и распаковки файлов

In [1]:
pip install pandas-profiling[notebook,html]

In [2]:
import math
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from numpy.random import RandomState
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.model_selection import GridSearchCV
import sweetviz as sv
import pandas_profiling
from pandas_profiling import ProfileReport
from scipy import stats as st
from pathlib import Path
path = Path.cwd()

In [3]:
#распакуем файлы
df_0 = pd.read_csv('datasets/geo_data_0.csv')
df_1 = pd.read_csv('datasets/geo_data_1.csv')
df_2 = pd.read_csv('datasets/geo_data_2.csv')

Изучим данные с помощью pandas-profiling

In [4]:
pandas_profiling.ProfileReport(df_0)

In [5]:
pandas_profiling.ProfileReport(df_1)

In [6]:
pandas_profiling.ProfileReport(df_2)

Посмотрим, что за неуникальные id находятся в столбце `id`

In [7]:
df_0[df_0['id'].isin(['fiKDv', 'QcMuo'])]


Удалим эти неуникальные id

In [8]:
df_0 = df_0.drop_duplicates(subset=['id'], keep='first').reset_index(drop=True)
df_1 = df_1.drop_duplicates(subset=['id'], keep='first').reset_index(drop=True)
df_2 = df_2.drop_duplicates(subset=['id'], keep='first').reset_index(drop=True)

Проверим

In [9]:
df_0[df_0['id'].isin(['fiKDv', 'QcMuo'])]

**Вывод**

При изучении всех трёх файлов выявлено, что пропуски  и дубликаты в данных отсутствуют. Есть несколько неуникальных id во всех трёх регионах, но признаки у них разные, возможно, что либо техническая ошибка при записи, либо специально делались замеры дважды. Т.к. количество этих неуникальных id в каждом регионе ничтожно мало, то просто удалили их, чтобы они не мешали дальнейшему исследованию.


В датасете df_0 признаки f0 и f1(в отличии от признака f2), а так же в датасете df_1 признак f0 не имеют нормального распределения. Возможно, данные признаки зависят от процесса, который постоянно меняется. Медианы и средние в двух данных регионах по признаку `product` схожи (91.85 и 92.5, 94.95 и 95 соответственно).

В датасете df_2  все признаки имеют нормальное распределение. 

 Данные региона df_1 по объёму запасов (столбец `product`) в скважине (тыс. баррелей) сильно отличается от других регионов. f2 и `product` имеют очень высокую корреляцию, так как уникальных значений всего 12 (возможно, что только 12, но очень перспективных скважин?) и f2 имеет маленький разброс значений от 0 до 5. Т.к. регион df_1 сильно выбивается от двух других регионов по данным с 12ю уникальными значениями, то смею предположить, что, возможно, данные собирались иначе, с другим оборудованием либо по другой схеме. Медина и среднее по признаку `product`: 57.08 и 68.83 соответственно.

### Обучение и проверка модели для каждого региона:

#### Разбивка данных на обучающую и валидационную выборки в соотношении 75:25.


Создадим функции для обучения модели линейной регрессии и вывода среднего всего фактического сырья
и среднего предсказанного сырья

In [10]:
def make_predictictions(data):
    features = data.drop(['id', 'product'], axis=1)
    target = data['product']
    features_train, features_valid, target_train, target_valid = train_test_split(
        features, target, test_size=.25, random_state=12345)
    model = LinearRegression()
    model.fit(features_train, target_train)
    return pd.Series(model.predict(features_valid)), target_valid

In [11]:
predicted_0, target_0 = make_predictictions(df_0)
predicted_1, target_1 = make_predictictions(df_1)
predicted_2, target_2 = make_predictictions(df_2)

In [12]:
def show_rmse(answers, predictions):
    mse = mean_squared_error(answers, predictions)
    r2 = round(r2_score(answers, predictions), 4)
    print('Реальное среднее значение, тыс. баррелей:', answers.mean())
    print('Предсказанное среднее значение модели, тыс. баррелей:', predictions.mean())
    print('RMSE модели:', mse ** 0.5)
    print('Коэффициент детерминации R2 модели:', r2)

In [13]:
print('Регион 0')
show_rmse(target_0, predicted_0)

In [14]:
print('Регион 1')
show_rmse(target_1, predicted_1)

In [15]:
print('Регион 2')
show_rmse(target_2, predicted_2)

**Вывод**

Данные фактического  количества нефти по сумме примерно совпадают по сумме с предсказанными значениями. Лучше всего модель предсказала запасы в регионе_1 (RMSE - 0.89), далее в регион_0 и регион_2 соответственно. 

Это потверждает коэффициент детерминации R2.  Чем R2 ближе к единице, тем лучше.

### Подготовка к расчёту прибыли

Сохраним все ключевые значения для расчётов в отдельных переменных.



При разведке региона исследуют 500 точек, из которых с помощью машинного обучения выбирают 200 лучших для разработки.

In [16]:
COUNT_OF_WELLS = 500  
BEST_WELLS = 200

Бюджет на разработку скважин в регионе — 10 млрд рублей.

In [17]:
BUDGET = 10_000_000_000

Цена разработки одной скважины — 50 млн рублей.

In [18]:
well_price = BUDGET / BEST_WELLS

При нынешних ценах один баррель сырья приносит 450 рублей дохода. Доход с каждой единицы продукта составляет 450 тыс. рублей, поскольку объём указан в тысячах баррелей.

In [19]:
PRICE_ONE_THOUSAND_BARRELS = 450_000

Рассчитаем достаточный объём сырья для безубыточной разработки новой скважины.

In [20]:
necessary_product = well_price / PRICE_ONE_THOUSAND_BARRELS
print('Необходимый объём сырья на одну скважину для безубыточной разработки новой скважины: {:.2f} тыс. баррелей'.
      format(necessary_product))

In [21]:
print("Средний истинный запас сырья в регионах, тыс. баррелей:")
print()
count = -1
targets = [target_0, target_1, target_2]
for target in targets:
    count += 1
    print('Регион ', count, ': ',  target.mean())
    if target.mean() < necessary_product:
        print(
            f'Средний предсказанный запас сырья в регионе меньше объёма достаточного'
            f' для безубыточной разработки на {round(necessary_product - target.mean(), 2)} тыс. баррелей')
    else:
        print(
            f'Средний предсказанный запас сырья в регионе  больше объёма достаточного'
            f' для безубыточной разработки на {round(target.mean() - necessary_product, 2)} тыс. баррелей')

Создадим функцию для расчета прибыли

In [22]:
def profit(target, probabilities):
    target = pd.Series(target).reset_index(drop=True)
    probabilities = pd.Series(probabilities).reset_index(drop=True)
    probs_sorted = probabilities.sort_values(ascending=False)
    selected = target[probs_sorted.index][:BEST_WELLS]
    mlrd = 1000000000
    return ((PRICE_ONE_THOUSAND_BARRELS * selected.sum()) - BUDGET) / mlrd

In [23]:
print('Регион 0')
profit(target_0, predicted_0)

In [24]:
print('Регион 1')
profit(target_1, predicted_1)

In [25]:
print('Регион 2')
profit(target_2, predicted_2)

**Вывод**

Самый прибыльный регион - Регион 0, так как может принести прибыль больше трех миллиардов рублей. Но средний запас сырья в скважинах меньше, чем точка безубыточности. Возможно, что, если рассмотреть не все 500 скважин, а только 200 лучших, риск убытка будет меньше. 

### Расчет рисков и прибыли для каждого региона


Посчитаем риски и прибыль для каждого региона. Найдем распределение прибыли с помощью техники Bootstrap.

In [26]:
BOOTHSTR_SAMPLS = 1000
RND = np.random.RandomState(12345)


def bootstrap(target, predictions):
    values = []
    for i in range(BOOTHSTR_SAMPLS):
        target_subsample = target.reset_index(drop=True).sample(
            n=COUNT_OF_WELLS, replace=True, random_state=RND)
        probs_subsample = predictions[target_subsample.index]
        rev = profit(target_subsample, probs_subsample)
        values.append(rev)

    values = pd.Series(values)
    risk = (values < 0).mean()
    lower = values.quantile(0.025)
    upper = values.quantile(0.975)
    mean = values.mean()

    print('Средняя прибыль по региону = {:.2f} млрд.руб.'.format(mean))
    print('95% доверительный интервал от {:.2f} до {:.2f} млрд.руб.'.format(
        lower, upper))

    print('Процент риска {:.2%}'.format(risk))

In [27]:
print('Регион 0')
bootstrap(target_0, predicted_0)

In [28]:
print('Регион 1')
bootstrap(target_1, predicted_1)

In [29]:
print('Регион 2')
bootstrap(target_2, predicted_2)

**Вывод**

Согласно прогнозам, сделанным с помощью техники Bootstrap, наиболее прибыльным будет разработка месторождений в Регионе_1 у него самые высокие показатели средней выручки и 95%-ного доверительного интервала. Также у этого региона самый низкий риск убытков. Кроме того доверительный интервал показывает, что даже при не очень удачном выборе разработка скважин окупится

### Общий вывод

In [30]:
rezult = {'RMSE': np.array([37.85, 0.89, 40.08]), 'R2': np.array(
    [0.28, 0.99, 0.20]), 'Среднее фактическое, тыс.баррелей': np.array(
    [92.16, 69.19, 94.79]), 'Среднее предсказанное, тыс.баррелей': np.array(
    [92.79, 69.18, 94.87]), 'Безубыточное отклонение, %': np.array(
    [20.56, 37.74, 14.70]), 'Прибыль, млрд.руб ': np.array(
    [0.38, 0.48, 0.32]),  'Риск, %': np.array(
    [7.70, 0.60, 12.40])}
rezult_df = pd.DataFrame(
    rezult, index=['Регион_0', 'Регион_1', 'Регион_2'])
rezult_df

Целью исследования было определить в каком из трех регионов бурить новые скважины. В нашем распоряжении были предоставлены пробы нефти в трёх регионах: в каждом 100 000 месторождений, где измерили качество нефти и объём её запасов.

На этапе подготовки данных было выявлено, что пропуски и дубликаты в датасетах отсутствуют. Изучили распределение, выяснили, что медианы и средние в двух  регионах (0 и 2) по признаку `product` схожи (91.85 и 92.5, 94.95 и 95 соответственно), а Регион_1 сильно выбивается от двух других регионов по данным с 12ю уникальными значениями, и предположили, что, возможно, данные собирались иначе, с другим оборудованием либо по другой схеме. Медиана и среднее по признаку `product`: 57.08 и 68.83 соответственно.

 Затем создали функции для обучения модели линейной регрессии и вывели среднее всего фактического сырья и среднее  предсказанного сырья. Выяснили, что данные фактического  количества нефти по сумме примерно совпадают по сумме с предсказанными значениями. Лучше всего модель предсказала запасы в регионе_1 (RMSE - 0.89), далее в Регион_0 и Регион_2 соответственно. Самым безубыточным регионом  при разработке всех 500 скважин показал себя Регион_2 (14.70%), далее Регион_0 и Регион_1 соответственно.
 
 Составили функцию расчета прибыли в регионе. Посчитали прибыль и риски за каждый регион на 200 самых перспективных скважин

После оценки рисков нужно было оставить лишь те регионы, в которых вероятность убытков меньше 2.5%. Такому условию удовлетворяет лишь один регион - Регион_1.